In [55]:
import json, pandas

In [56]:
trades = pandas.read_csv('20210420_vale3_bbt.log.csv',sep=';')
trades.head()

,time,price,qty,buyer,seller,agressor
0,1618929356,106.07,100,386,3,A
1,1618929356,106.07,100,386,85,A
2,1618929354,106.07,200,386,85,A
3,1618929349,106.07,100,3701,120,A
4,1618929348,106.04,300,72,40,V


In [57]:
broker = None
with open('broker.json', "r") as broker_json_file:
    broker = json.load(broker_json_file)

In [58]:
pnl={}
for i,trade in trades.iterrows():
    buyer = trade['buyer']
    seller = trade['seller']
    qty = trade['qty']
    price = trade['price']
    volume = price * qty
    agressor_buyer = 1 if trade['agressor'] == 'A' else 0
    agressor_seller = 1 if trade['agressor'] == 'V' else 0
    buyer_stats_from_dict = pnl.get(buyer)
    if buyer_stats_from_dict is None:
        stats = {}
        stats['net'] = volume
        stats['qty'] = qty
        stats['agressor'] = agressor_buyer
        pnl[buyer] = stats
    else:
        buyer_stats_from_dict['net'] = buyer_stats_from_dict['net'] + volume
        buyer_stats_from_dict['qty'] = buyer_stats_from_dict['qty'] + qty
        buyer_stats_from_dict['agressor'] = buyer_stats_from_dict['agressor'] + agressor_buyer

        pnl[buyer] = buyer_stats_from_dict
        
    seller_stats_from_dict = pnl.get(seller)
    if seller_stats_from_dict is None:
        stats = {}
        stats['net'] = -volume
        stats['qty'] = qty
        stats['agressor'] = agressor_seller
        pnl[seller] = stats
    else:
        seller_stats_from_dict['net'] = seller_stats_from_dict['net'] - volume
        seller_stats_from_dict['qty'] = seller_stats_from_dict['qty'] + qty
        seller_stats_from_dict['agressor'] = seller_stats_from_dict['agressor'] + agressor_seller

        pnl[seller] = seller_stats_from_dict

statistics_array = []
for idx in pnl:
    broker_name = [x for x in broker if x['playerMarketCode'] == str(idx)][1]['playerNameFull']
    statistics_array.append([idx, broker_name, pnl[idx]['net'], pnl[idx]['qty'], pnl[idx]['agressor']])
statistics = pandas.DataFrame(statistics_array, columns=['BROKER_ID', 'BROKER_NAME', 'NET', 'QTY', 'AGRESSOR'])

In [59]:
statistics.sort_values('QTY',ascending=False)

,BROKER_ID,BROKER_NAME,NET,QTY,AGRESSOR
14,8,UBS Brasil. CCTVM,18747882.0,2765200,2301
1,3,XP INVESTIMENTOS CCTVM S.A,-52130499.0,2330000,2925
28,13,MERRILL LYNCH S.A. CTVM,-25250738.0,2119500,614
4,120,BRASIL PLURAL,-1282870.0,1462400,1454
8,1618,IDEAL,9834157.0,1289700,1266
12,45,CREDIT SUISSE BRASIL S.A. CTVM,91518926.0,1216700,4122
2,85,BTG PACTUAL CTVM S.A,41743156.0,1074500,1004
6,40,MORGAN STANLEY D. W. CTVM S.A.,-32823650.0,971200,859
24,77,CITIGROUP,-55768921.0,865100,693
5,72,BRADESCO S.A. CTVM,-10668241.0,630900,634
